This script is to add data general. 
Import all required tools

In [1]:
import ixmp
import message_ix
import numpy as np
import pandas as pd
import yaml

from collections.abc import Mapping
from itertools import repeat
from message_ix.models import MESSAGE_ITEMS
from message_ix.utils import make_df
from message_ix.tools.get_report import get_report
from message_ix.tools.add_dac import add_dac

%matplotlib inline

mp = ixmp.Platform()

<IPython.core.display.Javascript object>

In [ ]:
#df_scenario = mp.scenario_list()
#df_scenario.to_excel('scenarios.xlsx')

This part is just to generate a dummy scenario as basis for adding technology parameters. Need to be removed once integrated

In [2]:
#base = message_ix.Scenario(mp, model='GENIE_sandbox', scenario="baseline")
#base = message_ix.Scenario(mp, model='GENIE_sandbox', scenario="daccsNoBound")
scen = message_ix.Scenario(mp, model='GENIE_sandbox', scenario="daccs1000")


Existing index sets of 'EMISSION_EQUIVALENCE' ['node', 'emission', 'type_tec', 'year'] do not match []


In [ ]:
#scen = base.clone("GENIE_sandbox", "daccsNoBound", "reporting test no CO2 bound", keep_solution=False)
#scen = base.clone("GENIE_sandbox", "daccs1000_nodaccs", "reporting test", keep_solution=False)
scen = base.clone("GENIE_sandbox", "daccs1000", "reporting test", keep_solution=False)
scen.check_out()

year_df = scen.vintage_and_active_years()
vintage_years, act_years = year_df["year_vtg"], year_df["year_act"]
model_horizon = scen.set("year")

In [ ]:
# adding DACCS
add_dac(scen)

In [ ]:
# add emission bound
scen.add_par('bound_emission', ['World', 'TCE_CO2', 'all', 'cumulative'], value=2421.17, unit="tC")

In [ ]:
scen.commit(comment="Copy and run daccs1000 budget with daccs scenario")


In [ ]:
scen.set_as_default()


In [ ]:
scen.solve(solve_options={'barcrossalg': '2','scaind':'1'})

# Before running report, check in GDX if results show daccs deployment

In [ ]:
from message_data.tools.post_processing.iamc_report_hackathon import report as reporting
reporting(
    mp,
    scen,
    "False",
    scen.model,
    scen.scenario,
    merge_hist=True,
    merge_ts=True,
    run_config="SSP_dev_run_config.yaml",
    #run_config="ENGAGE_SSP2_v417_run_config.yaml",
)


Creating basic dataframe to be filled in later

In [3]:
get_report(scen,'DACs')

year_act  1960  1965  1970  1975  1980  1985  1990  1995  2000  2005  ...  \
node_loc                                                              ...   
R12_AFR    0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
R12_CHN    0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
R12_EEU    0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
R12_FSU    0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
R12_LAM    0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
R12_MEA    0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
R12_NAM    0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
R12_PAO    0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
R12_PAS    0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
R12_RCPA   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
R12_SAS    0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   

In [4]:
mp.close_db()